In [ ]:
# Install the required packages
%pip install -qU google-generativeai klavis requests


In [ ]:
import os
import json
import google.generativeai as genai
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"  # Replace with your actual Google AI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

# Configure Gemini
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [ ]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}, and the instance id is {youtube_mcp_instance.instance_id}")


In [ ]:
def gemini_with_mcp_server(mcp_server_url: str, user_query: str):
    # Get tools from MCP server
    mcp_server_tools = klavis_client.mcp_server.list_tools(
        server_url=mcp_server_url,
        connection_type=ConnectionType.STREAMABLE_HTTP,
        format=ToolFormat.GOOGLE,
    )
    
    # Initialize Gemini model with tools
    model = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        tools=mcp_server_tools.tools
    )
    
    # Start chat
    chat = model.start_chat()
    
    # Send initial message
    response = chat.send_message(user_query)
    
    # Check if function call is requested
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        function_name = function_call.name
        function_args = dict(function_call.args)
        
        print(f"🔧 Calling: {function_name}, with args: {function_args}")
        
        # Call the MCP server tool
        result = klavis_client.mcp_server.call_tools(
            server_url=mcp_server_url,
            tool_name=function_name,
            tool_args=function_args,
            connection_type=ConnectionType.STREAMABLE_HTTP
        )
        
        # Send function response back to model
        function_response = genai.protos.Part(
            function_response=genai.protos.FunctionResponse(
                name=function_name,
                response={"result": result.result.content[0]['text']}
            )
        )
        
        final_response = chat.send_message([function_response])
        return final_response.text
    else:
        return response.text


In [ ]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick a video you like!

result = gemini_with_mcp_server(
    mcp_server_url=youtube_mcp_instance.server_url, 
    user_query=f"Please provide a complete summary of this YouTube video with timestamp: {YOUTUBE_VIDEO_URL}"
)

print(result)


In [ ]:
import webbrowser

gmail_mcp_server = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

webbrowser.open(gmail_mcp_server.oauth_url)

print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_server.oauth_url}")


In [ ]:
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # Replace with your email
EMAIL_SUBJECT = "Test Gemini + Gmail MCP Server"
EMAIL_BODY = "Hello World from Gemini!"

result = gemini_with_mcp_server(
    mcp_server_url=gmail_mcp_server.server_url, 
    user_query=f"Please send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(result)
